In [3]:
import re
import json
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

# 1. NS Shop 상품 카테고리 크롤링

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
}

In [7]:
def getCategory():
    url = 'http://www.nsmall.com/CategoryDisplay?cate1Code=1583596&cate2Code=1583567&storeId=13001&langId=-9&catalogId=97001&menuUri=NSCategorySubDisplay&categoryId=1583567'
    response = requests.get(url, headers = headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    category = list(map(lambda x : x.text, soup.select('div.category_layer_wrapper > div > h3')))
    items = soup.select('li.cate_item')
    total = []
    for i, item in enumerate(items):
        if i  < 3:
            cate = category[0]
        elif i < 8:
            cate = category[1]
        else:
            cate = category[2]

        cate_code = str(item.select_one('a')).split('cate1Code=')[1].split('&')[0]
        main_code = str(item.select_one('a')).split('cate2Code=')[1].split('&')[0]
        main = item.select_one('a > span').text

        raw_sub = item.select('div.low_depth_wrap > ul > li')

        value = [[cate_code, cate, main_code, main, s['id'], s.select_one('a > span').text] for s in raw_sub]
        total.extend(value)
    return total

In [8]:
category = getCategory()

In [9]:
ns_category = pd.DataFrame(category, columns = ['cate1Code', 'cate1', 'cate2Code', 'cate2', 'mainCode', 'main'])

In [10]:
ns_category.head()

,cate1Code,cate1,cate2Code,cate2,mainCode,main
0,1583595,푸드/헬스,1583566,가공식품,1583608,간편/즉석요리
1,1583595,푸드/헬스,1583566,가공식품,1583609,과자/간식
2,1583595,푸드/헬스,1583566,가공식품,1583610,김치/반찬
3,1583595,푸드/헬스,1583566,가공식품,1583611,라면/면/스프
4,1583595,푸드/헬스,1583566,가공식품,1583612,조미료/소스/장류


In [12]:
total = []
for cate1Code, cate2Code in tqdm(ns_category[['cate1Code', 'cate2Code']].drop_duplicates().values):
    url = f'http://www.nsmall.com/CategoryDisplay?cate1Code={cate1Code}&cate2Code={cate2Code}&storeId=13001&langId=-9&catalogId=97001&menuUri=NSCategorySubDisplay&categoryId={cate2Code}'
    response = requests.get(url, headers = headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    for sub in soup.select('div.depth_2'):
        value = [[str(s).split('cate3Code=')[1][:7], str(s).split('categoryId=')[1][:7], s.text] for s in sub.select('ul > li > a')]
        total.extend(value)

100%|██████████| 13/13 [00:19<00:00,  1.53s/it]


In [13]:
len(ns_category), len(total)

(67, 425)

In [14]:
ns_category = ns_category.merge(pd.DataFrame(total, columns = ['mainCode', 'subCode', 'sub']), on = 'mainCode', how = 'right').sort_values('cate2')

In [ ]:
ns_category.to_excel(os.path.join('..', '0.Data', '01_제공데이터', 'ns_shop_category.xlsx'), index = False)

In [16]:
ns_category.head(50)

,cate1Code,cate1,cate2Code,cate2,mainCode,main,subCode,sub
0,1583595,푸드/헬스,1583566,가공식품,1583608,간편/즉석요리,1585836,간편조리
22,1583595,푸드/헬스,1583566,가공식품,1583614,탕/국/찌개,1583091,찌개
21,1583595,푸드/헬스,1583566,가공식품,1583613,커피/생수/음료,1585762,탄산음료/주스
20,1583595,푸드/헬스,1583566,가공식품,1583613,커피/생수/음료,1583094,커피
19,1583595,푸드/헬스,1583566,가공식품,1583613,커피/생수/음료,1583095,차/티백
18,1583595,푸드/헬스,1583566,가공식품,1583613,커피/생수/음료,1585782,전통주
17,1583595,푸드/헬스,1583566,가공식품,1583613,커피/생수/음료,1585853,생수
16,1583595,푸드/헬스,1583566,가공식품,1583613,커피/생수/음료,1585746,두유/우유
15,1583595,푸드/헬스,1583566,가공식품,1583612,조미료/소스/장류,1585593,조미료/양념
14,1583595,푸드/헬스,1583566,가공식품,1583612,조미료/소스/장류,1585626,장류


In [31]:
ns_category['cate1'].unique()

array(['푸드/헬스', '홈/라이프스타일', '패션/뷰티'], dtype=object)

In [24]:
ns_category['cate2'].unique()

array(['가공식품', '가구/침구/인테리어', '가전/디지털', '건강/다이어트식품', '농수축/신선식품', '반려동물',
       '뷰티', '스포츠/레저', '여행/하나투어/렌탈', '유아동', '잡화/슈즈/쥬얼리', '주방/생활/건강',
       '패션의류'], dtype=object)

In [34]:
ns_category[ns_category['cate2'] == '스포츠/레저'].head(50)

,cate1Code,cate1,cate2Code,cate2,mainCode,main,subCode,sub
234,1583596,홈/라이프스타일,1583571,스포츠/레저,1583632,헬스/요가/수영/스키,1590660,요가복
233,1583596,홈/라이프스타일,1583571,스포츠/레저,1583632,헬스/요가/수영/스키,1590666,요가매트
232,1583596,홈/라이프스타일,1583571,스포츠/레저,1583632,헬스/요가/수영/스키,1590624,승마운동기
231,1583596,홈/라이프스타일,1583571,스포츠/레저,1583632,헬스/요가/수영/스키,1590664,스트레칭
228,1583596,홈/라이프스타일,1583571,스포츠/레저,1583632,헬스/요가/수영/스키,1590680,수영용품
229,1583596,홈/라이프스타일,1583571,스포츠/레저,1583632,헬스/요가/수영/스키,1590701,스키/스노우보드
226,1583596,홈/라이프스타일,1583571,스포츠/레저,1583632,헬스/요가/수영/스키,1590640,복근운동기구
227,1583596,홈/라이프스타일,1583571,스포츠/레저,1583632,헬스/요가/수영/스키,1590674,수영복
235,1583596,홈/라이프스타일,1583571,스포츠/레저,1583632,헬스/요가/수영/스키,1590662,요가용품
230,1583596,홈/라이프스타일,1583571,스포츠/레저,1583632,헬스/요가/수영/스키,1590632,스텝퍼


# 2. 한국은행 경제통제시스템 API 크롤링

In [ ]:
key = input()

In [6]:
meta_url = f'http://ecos.bok.or.kr/api/StatisticTableList/{key}/json/kr/1/10000/'

In [7]:
response = requests.get(meta_url)

In [10]:
meta = pd.json_normalize(response.json()['StatisticTableList']['row'])

In [12]:
meta.loc[meta['STAT_NAME'].str.contains('신용카드')]

,SRCH_YN,ORG_NAME,CYCLE,STAT_NAME,STAT_CODE,P_STAT_CODE
136,N,,,5.8.신용카드,043Y700,000Y009
137,Y,,MM,5.8.1 신용카드,043Y013,043Y700
138,Y,,MM,5.8.2 지역별 소비유형별 개인 신용카드,043Y070,043Y700


In [32]:
credit_url = f'http://ecos.bok.or.kr/api/StatisticSearch/{key}/json/en/1/30000/043Y070/MM/202001/202008'

In [33]:
response = requests.get(credit_url)

In [34]:
credit = pd.json_normalize(response.json()['StatisticSearch']['row'])

In [35]:
credit['TIME'].value_counts().sort_index()

202001    1476
202002    1476
202003    1476
202004    1476
202005    1476
Name: TIME, dtype: int64